In [1]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠙ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest ⠋ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest ⠙ pulling manifest 
pulling 970aa74c0a90... 100% ▕███████████████

In [ ]:
!sudo docker run -d   --name qdrant-container   -p 6333:6333   -v $(pwd)/qdrant_data:/qdrant/storage   qdrant/qdrant

## Old

In [9]:
from langchain_community.embeddings import OllamaEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.http import models
import faiss
import pickle
import numpy as np
import pandas as pd

In [10]:
with open('metadata_with_chunck_embeddings.pkl', 'rb') as file:
    data = pickle.load(file)

In [12]:
data[0]

{'_id': '63927168098523fa9d5ef047',
 'title': 'Are The Democrats Screwed In The Senate After 2024?',
 'author': 'Nate Silver',
 'publish_date': Timestamp('1970-01-01 00:27:50.515535'),
 'full_text': '\n\n\n\n2028 Election\nAre The Democrats Screwed In The Senate After 2024?\nNo, but the party faces an uphill battle.\n\n\n\nBy Nate Silver\n\n\nDec. 8, 2022, at 11:05 AM\t\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSamuel Corum / Getty Images\n\n\n\n\nI’m not that impressed by long-term projections of Democratic doom in the Senate. Mostly that’s because I’m not that impressed by long-term political projections in general. Political coalitions change; not many people would have had it on their bingo card that Democrats would obtain a Senate majority in 2020 by winning two runoffs in Georgia, and then would actually expand on that majority in 2022 following another runoff in Georgia despite their party controlling the White House.\nWith that said, while I don’t think we can say very much about what

In [13]:
chunk_embeddings = []
for doc in data:
    chunk_embeddings.extend(doc['chunk_embeddings'])

In [14]:
chunk_embeddings = np.array(chunk_embeddings)

In [15]:
type(chunk_embeddings)

numpy.ndarray

In [16]:
model = OllamaEmbeddings(model='nomic-embed-text')

In [17]:
d = chunk_embeddings.shape[1]

In [18]:
index = faiss.IndexHNSWFlat(d, 64)

In [19]:
index.is_trained

True

In [20]:
index.add(chunk_embeddings)

In [21]:
faiss.write_index(index, "faiss_index.index")

In [22]:
index = faiss.read_index("faiss_index.index")

In [25]:
client = QdrantClient("http://localhost:6333")
collection_name = "documents_collection"

In [26]:
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(size=d, distance=models.Distance.COSINE),
)

/tmp/ipykernel_14226/1638598554.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [27]:
points = []
for doc_id, doc in enumerate(data):
    for chunk_id, embedding in enumerate(doc['chunk_embeddings']):
        points.append(
            models.PointStruct(
                id=len(points),  # Ensure unique ID for each chunk
                vector=embedding,
                payload={
                    "author": doc['author'],
                    "title": doc['title'],
                    "publish_date": doc['publish_date'].isoformat(),
                    "full_text": doc['full_text'],
                    "url": doc['url'],
                }
            )
        )

In [28]:
batch_size = 32  

for i in range(0, len(points), batch_size):
    batch = points[i:i + batch_size]
    try:
        client.upsert(collection_name=collection_name, points=batch)
    except Exception as e:
        print(f"Error during batch upsert: {e}")
        break

In [29]:
query = 'Harvard and UNC affirmative-action cases last term'
query = model.embed_query(query)
query_embedding = np.array([query])

In [30]:
k = 50
distances, indices = index.search(query_embedding, k)

In [31]:
result_ids = indices[0].tolist()

In [32]:
results = client.retrieve(collection_name=collection_name, ids=result_ids)

In [33]:
len(results)

50

In [34]:
top_3_results = results[:3]

In [35]:
for result in top_3_results:
    print(f"Author: {result.payload['author']}")
    print(f"Title: {result.payload['title']}")
    print(f"Publish Date: {result.payload['publish_date']}")
    print(f"Full Text: {result.payload['full_text']}")  
    print(f"URL: {result.payload['url']}")
    print("=="*50)

Author: Harvard Gazette
Title: Opportunity to vote for Board of Overseers, HAA elected directors
Publish Date: 1970-01-01T00:27:56.476804
Full Text: This spring, Harvard degree holders will have the opportunity to vote for new members of the Harvard Board of Overseers and elected directors of the Harvard Alumni Association (HAA).
The elections will begin March 31. Eligible voters can vote either online or by paper ballot. Completed ballots must be received by 5 p.m. (EDT) on May 16. All holders of Harvard degrees as of Jan. 1, except officers of instruction and government at Harvard and members of the Harvard Corporation, are entitled to vote for Overseer candidates. All Harvard degree holders as of Jan. 1 may vote for HAA elected directors.
The candidates listed below have been named by the nominating committee appointed by the Harvard Alumni Association’s volunteer leadership.  They will be considered by voters for five anticipated vacancies on the Board of Overseers and for six open

In [36]:
def cosine_similarity(vector_a, vector_b):
    """Compute the cosine similarity between two vectors."""
    vector_b = np.squeeze(vector_b)  # Ensure both vectors have the shape (768,)
    dot_product = np.dot(vector_a, vector_b)
    norm_a = np.linalg.norm(vector_a)
    norm_b = np.linalg.norm(vector_b)
    return dot_product / (norm_a * norm_b)

In [37]:
def rerank(results, query_embedding):
    """
    Rerank the top results based on a custom criterion.
    
    Parameters:
    - results: List of search results to rerank.
    - query_embedding: The embedding of the original query.

    Returns:
    - reranked_results: List of reranked results.
    """
    # Example: Using cosine similarity for reranking
    reranked_results = sorted(
        results,
        key=lambda result: cosine_similarity(result['embedding'], query_embedding),
        reverse=True  # Higher scores first
    )
    return reranked_results

In [38]:
mutable_results = []
for result in top_3_results:
    # Wrap the Record object and its embedding in a dictionary
    embedding = model.embed_query(result.payload['full_text'])  # Generate the embedding for reranking
    mutable_result = {
        "record": result,  # The original Record object
        "embedding": embedding,  # The embedding for reranking
    }
    mutable_results.append(mutable_result)

In [39]:
query_embedding = np.squeeze(query_embedding) 

In [40]:
reranked_results = rerank(mutable_results, query_embedding)

In [41]:
# Display the reranked results
for result in reranked_results:
    record = result['record']  # This is the original Record object
    print(f"Author: {record.payload['author']}")
    print(f"Title: {record.payload['title']}")
    print(f"Publish Date: {record.payload['publish_date']}")
    print(f"Full Text: {record.payload['full_text']}")  # Truncate long text for brevity
    print(f"URL: {record.payload['url']}")
    print(f"Cosine Similarity: {cosine_similarity(result['embedding'], query_embedding)}")
    print("=="*50)

Author: Robby Soave
Title: How Affirmative Action Lost at the Supreme Court
Publish Date: 1970-01-01T00:28:14.080831
Full Text: 






The end of affirmative action in university admissions has been prophesied since 2003, when the Supreme Court issued its decision in Grutter v. Bollinger. In the majority opinion, Justice Sandra Day O'Connor wrote that "25 years from now, the use of racial preferences will no longer be necessary to further the interest approved today." That reckoning has now arrived, and five years earlier than predicted: In June, the Supreme Court ruled 6–3 that public universities must stop favoring certain applicants, and disfavoring others, based on their race or ethnicity. "Eliminating racial discrimination means eliminating all of it," Chief Justice John Roberts declared, writing for the majority in Students for Fair Admissions v. President and Fellows of Harvard College. "In other words, the student must be treated based on his or her experiences as an individual

In [42]:
!ollama pull llama3

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕               

## New

In [43]:
from qdrant_client import QdrantClient
from langchain_community.embeddings import OllamaEmbeddings
import faiss
from langchain_community.llms import Ollama
import numpy as np
import markdown
from IPython.display import display, HTML

In [44]:
index = faiss.read_index("faiss_index.index")

In [45]:
client = QdrantClient(url="http://localhost:6333")

In [46]:
embeddings = OllamaEmbeddings(model='nomic-embed-text')

In [47]:
query = 'Harvard and UNC affirmative-action cases last term'
query = embeddings.embed_query(query)
query_embedding = np.array([query])

In [48]:
k = 50
distances, indices = index.search(query_embedding, k)

In [49]:
result_ids = indices[0].tolist()

In [50]:
results = client.retrieve(collection_name="documents_collection", ids=result_ids)

In [51]:
top_3_results = results[:3]

In [52]:
def cosine_similarity(vector_a, vector_b):
    """Compute the cosine similarity between two vectors."""
    vector_b = np.squeeze(vector_b)  # Ensure both vectors have the shape (768,)
    dot_product = np.dot(vector_a, vector_b)
    norm_a = np.linalg.norm(vector_a)
    norm_b = np.linalg.norm(vector_b)
    return dot_product / (norm_a * norm_b)

In [53]:
def rerank(results, query_embedding):
    """
    Rerank the top results based on a custom criterion.
    
    Parameters:
    - results: List of search results to rerank.
    - query_embedding: The embedding of the original query.

    Returns:
    - reranked_results: List of reranked results.
    """
    # Example: Using cosine similarity for reranking
    reranked_results = sorted(
        results,
        key=lambda result: cosine_similarity(result['embedding'], query_embedding),
        reverse=True  # Higher scores first
    )
    return reranked_results

In [54]:
mutable_results = []
for result in top_3_results:
    # Wrap the Record object and its embedding in a dictionary
    embedding = embeddings.embed_query(result.payload['full_text'])  # Generate the embedding for reranking
    mutable_result = {
        "record": result,  # The original Record object
        "embedding": embedding,  # The embedding for reranking
    }
    mutable_results.append(mutable_result)

In [55]:
query_embedding = np.squeeze(query_embedding) 

In [56]:
reranked_results = rerank(mutable_results, query_embedding)

In [57]:
# Initialize a list to hold the formatted results
saved_results = []

# Loop through the reranked results and format each one
for result in reranked_results:
    record = result['record']  # This is the original Record object
    
    # Create a formatted string for each result
    formatted_result = (
        f"Author: {record.payload['author']}\n"
        f"Title: {record.payload['title']}\n"
        f"Date: {record.payload['publish_date']}\n"
        f"Text: {record.payload['full_text'][:500]}...\n"  # Truncate long text for brevity
        f"URL: {record.payload['url']}\n"
    )
    
    # Append the formatted string to the results list
    saved_results.append(formatted_result)

In [58]:
saved_results

['Author: Robby Soave\nTitle: How Affirmative Action Lost at the Supreme Court\nDate: 1970-01-01T00:28:14.080831\nText: \n\n\n\n\n\n\nThe end of affirmative action in university admissions has been prophesied since 2003, when the Supreme Court issued its decision in Grutter v. Bollinger. In the majority opinion, Justice Sandra Day O\'Connor wrote that "25 years from now, the use of racial preferences will no longer be necessary to further the interest approved today." That reckoning has now arrived, and five years earlier than predicted: In June, the Supreme Court ruled 6–3 that public universities must stop favoring cer...\nURL: https://reason.com/2023/09/07/affirmative-action-loses-in-court/\n',
 'Author: Ilya Shapiro\nTitle: Amicus Brief: Coalition for TJ v. Fairfax County School District\nDate: 1970-01-01T00:28:15.311346\nText: A group of parents challenged the Fairfax County, Virginia school board’s revamping of admissions policies at the Thomas Jefferson High School for Science a

In [59]:
saved_results_combined = "\n".join(saved_results)
print(saved_results_combined)

Author: Robby Soave
Title: How Affirmative Action Lost at the Supreme Court
Date: 1970-01-01T00:28:14.080831
Text: 






The end of affirmative action in university admissions has been prophesied since 2003, when the Supreme Court issued its decision in Grutter v. Bollinger. In the majority opinion, Justice Sandra Day O'Connor wrote that "25 years from now, the use of racial preferences will no longer be necessary to further the interest approved today." That reckoning has now arrived, and five years earlier than predicted: In June, the Supreme Court ruled 6–3 that public universities must stop favoring cer...
URL: https://reason.com/2023/09/07/affirmative-action-loses-in-court/

Author: Ilya Shapiro
Title: Amicus Brief: Coalition for TJ v. Fairfax County School District
Date: 1970-01-01T00:28:15.311346
Text: A group of parents challenged the Fairfax County, Virginia school board’s revamping of admissions policies at the Thomas Jefferson High School for Science and Technology (“TJ”) a

---

In [60]:
# Initialize the LLaMA3 model using Ollama
llm = Ollama(model="llama3")

In [75]:
output_prompt = f"""
Please present the following top 3 search results in a clean and readable format. Each result should be clearly separated with headers and bullet points, and should be formatted as follows: 

Author: [Robby Soave](URL with him name)  
/n
   Title: TITLE_HERE  
/n
   Date: DATE_HERE  
/n
   Text: the text here, with including the full text. 
/n

{saved_results_combined}

Make sure that the author's name is hyperlinked to the corresponding URL, and that the date is formatted properly. Each Author's name should be bolded, and the entire response should be returned in Markdown format.

Make it as modren way

last thing write I hope this is what you were looking for!
"""


In [76]:
responsellama= llm.generate(
    prompts=[output_prompt],
    tags=["example_run"],
    metadata={"source": "user_query"}
)

In [77]:
print(responsellama)

generations=[[GenerationChunk(text='Here are the top 3 search results in a clean and readable format:\n\n**Top Search Results**\n\n### **1. How Affirmative Action Lost at the Supreme Court**\n**[Robby Soave](https://reason.com/20xx/09/07/affirmative-action-loses-in-court/)**\nDate: January 1, 1970T00:28:14.080831\n\n* Text: The end of affirmative action in university admissions has been prophesied since 2003, when the Supreme Court issued its decision in Grutter v. Bollinger. In the majority opinion, Justice Sandra Day O\'Connor wrote that "25 years from now, the use of racial preferences will no longer be necessary to further the interest approved today." That reckoning has now arrived, and five years earlier than predicted: In June, the Supreme Court ruled 6–3 that public universities must stop favoring certain groups.\n\n### **2. Amicus Brief: Coalition for TJ v. Fairfax County School District**\n**[Ilya Shapiro](https://manhattan.institute/article/amicus-brief-coalition-for-tj-v-fa

In [78]:
def render_markdown(md_text):
    html = markdown.markdown(md_text)
    display(HTML(html))

In [79]:
render_markdown(responsellama.generations[0][0].text)